### Visualize 1

In [ ]:
import pandas as pd
import numpy as np
import os
from pythreejs import *
from IPython.display import display
import ipywidgets as widgets
import asyncio

# Load data
body_names = ['Sun', 'Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']
data = []

for name in body_names:
    df = pd.read_csv(f'outputs/{name}.csv')
    data.append(df[['X', 'Y', 'Z']].to_numpy())

num_frames = len(data[0])
num_bodies = len(data)

# Create spheres
colors = ['yellow', 'gray', 'orange', 'blue', 'red', 'brown', 'gold', 'cyan', 'purple']
radii = [0.3, 0.05, 0.06, 0.07, 0.06, 0.2, 0.17, 0.12, 0.12]

spheres = []
for i in range(num_bodies):
    mat = MeshLambertMaterial(color=colors[i])
    geo = SphereBufferGeometry(radius=radii[i], widthSegments=16, heightSegments=16)
    mesh = Mesh(geometry=geo, material=mat, position=tuple(data[i][0]))
    spheres.append(mesh)

# Set up the scene
scene = Scene(children=[
    *spheres,
    AmbientLight(intensity=0.5),
    DirectionalLight(position=[5, 5, 5], intensity=0.6)
], background='black')

camera = PerspectiveCamera(position=[0, 0, 8], fov=60)
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=800, height=600)

# Animation state
is_playing = False
frame_index = 0
speed = 1  # frames per update

# Update function
def update_scene():
    global frame_index
    for i in range(num_bodies):
        if frame_index < len(data[i]):
            spheres[i].position = tuple(data[i][frame_index])
    frame_index = (frame_index + speed) % num_frames

# Async animation loop
async def animate_loop():
    global is_playing
    while is_playing:
        update_scene()
        await asyncio.sleep(0.03)

# Button handlers
def toggle_play(change):
    global is_playing
    is_playing = not is_playing
    play_button.description = 'Pause' if is_playing else 'Play'
    if is_playing:
        asyncio.ensure_future(animate_loop())

def reset_scene(change):
    global frame_index
    frame_index = 0
    update_scene()

def increase_speed(change):
    global speed
    speed = min(speed + 1, 10)

def decrease_speed(change):
    global speed
    speed = max(speed - 1, 1)

# Buttons
play_button = widgets.Button(description='Play')
play_button.on_click(toggle_play)

reset_button = widgets.Button(description='Reset')
reset_button.on_click(reset_scene)

faster_button = widgets.Button(description='Faster')
faster_button.on_click(increase_speed)

slower_button = widgets.Button(description='Slower')
slower_button.on_click(decrease_speed)

controls = widgets.HBox([play_button, reset_button, slower_button, faster_button])

# Display everything
display(renderer, controls)


### Visualize 2

In [ ]:
import pandas as pd
import numpy as np
import os
from pythreejs import *
from IPython.display import display
import ipywidgets as widgets
import asyncio

# Load data
body_names = ['Sun', 'Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']
colors = ['yellow', 'gray', 'orange', 'blue', 'red', 'brown', 'gold', 'cyan', 'purple']
radii = [0.3, 0.05, 0.06, 0.07, 0.06, 0.2, 0.17, 0.12, 0.12]

data = []
for name in body_names:
    df = pd.read_csv(f'outputs/{name}.csv')
    data.append(df[['X', 'Y', 'Z']].to_numpy())

num_frames = len(data[0])
num_bodies = len(data)

# Create spheres (planets and Sun)
spheres = []
for i in range(num_bodies):
    mat = MeshLambertMaterial(color=colors[i])
    geo = SphereBufferGeometry(radius=radii[i], widthSegments=16, heightSegments=16)
    mesh = Mesh(geometry=geo, material=mat, position=tuple(data[i][0]))
    spheres.append(mesh)

# Create orbit trails
orbit_trails = []
trail_length = 100
for i in range(1, num_bodies):  # Skip the Sun
    trail_pos = np.zeros((trail_length, 3))
    trail_geom = BufferGeometry(attributes={
        'position': BufferAttribute(trail_pos.copy(), normalized=False)
    })
    trail_line = Line(geometry=trail_geom,
                      material=LineBasicMaterial(color=colors[i], linewidth=1, transparent=True, opacity=0.6))
    orbit_trails.append(trail_line)

# Create dynamic trails (planet positions as trails)
planet_trails = []
for i in range(1, num_bodies):  # Skip the Sun
    trail_pos = np.zeros((trail_length, 3))
    trail_geom = BufferGeometry(attributes={
        'position': BufferAttribute(trail_pos.copy(), normalized=False)
    })
    trail_line = Line(geometry=trail_geom,
                      material=LineBasicMaterial(color=colors[i], linewidth=1, transparent=True, opacity=0.3))
    planet_trails.append(trail_line)

# Labels (2D HTML overlay widgets)
labels = []
label_widgets = widgets.HBox([])

for name in body_names:
    lbl = widgets.Label(value=name)
    lbl.layout.margin = '0 10px 0 10px'
    labels.append(lbl)

label_box = widgets.HBox(labels)
label_box.layout.justify_content = 'center'

# Scene setup
scene = Scene(children=[
    *spheres, *orbit_trails, *planet_trails,  # Include Sun and planets here
    AmbientLight(intensity=0.5),
    DirectionalLight(position=[5, 5, 5], intensity=0.6)
], background='black')

camera = PerspectiveCamera(position=[0, 0, 8], fov=60)
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=900, height=700)

# Animation state
is_playing = False
frame_index = 0
speed = 1  # frames per update

# Update function
def update_scene():
    global frame_index
    for i in range(num_bodies):  # Include Sun and planets here
        # Get the position of the current body
        if frame_index < len(data[i]):
            pos = data[i][frame_index]
            spheres[i].position = tuple(pos)

            # Update orbit trails (planets only, skip Sun)
            if i > 0:  # Only update for planets (skip Sun)
                orbit_data = data[i][:frame_index+1]  # Use all positions up to the current frame
                trail_data = orbit_data[-trail_length:]  # Get the last `trail_length` positions
                trail_array = np.zeros((trail_length, 3))
                trail_array[-len(trail_data):] = trail_data  # Fill with the last trail positions
                
                # Update the orbit line geometry with the trail positions
                orbit_trails[i-1].geometry.attributes['position'].array = trail_array
                orbit_trails[i-1].geometry.attributes['position'].needsUpdate = True

                # Update dynamic planet trails
                planet_trail_data = data[i][max(0, frame_index - trail_length):frame_index + 1]
                planet_trail_array = np.zeros((trail_length, 3))
                planet_trail_array[-len(planet_trail_data):] = planet_trail_data

                # Update the trail geometry for planets
                planet_trails[i-1].geometry.attributes['position'].array = planet_trail_array
                planet_trails[i-1].geometry.attributes['position'].needsUpdate = True

    # After updating the positions and trails for each body, render the scene
    renderer.render(scene, camera)
    
    # Move to the next frame (or loop back if at the end)
    frame_index = (frame_index + 1) % len(data[0])

# Async animation loop
async def animate_loop():
    global is_playing, frame_index
    while is_playing:
        for i in range(num_bodies):  # Include Sun and planets here
            if frame_index < len(data[i]):
                # Move sphere to new position
                spheres[i].position = tuple(data[i][frame_index])

        frame_index = (frame_index + speed) % num_frames
        await asyncio.sleep(0.03)

# Button handlers
def toggle_play(change):
    global is_playing
    is_playing = not is_playing
    play_button.description = 'Pause' if is_playing else 'Play'
    if is_playing:
        asyncio.ensure_future(animate_loop())

def reset_scene(change):
    global frame_index
    frame_index = 0
    update_scene()

def increase_speed(change):
    global speed
    speed = min(speed + 1, 10)

def decrease_speed(change):
    global speed
    speed = max(speed - 1, 1)

# Buttons
play_button = widgets.Button(description='Play')
play_button.on_click(toggle_play)

reset_button = widgets.Button(description='Reset')
reset_button.on_click(reset_scene)

faster_button = widgets.Button(description='Faster')
faster_button.on_click(increase_speed)

slower_button = widgets.Button(description='Slower')
slower_button.on_click(decrease_speed)

controls = widgets.HBox([play_button, reset_button, slower_button, faster_button])

# Display
display(renderer, controls, label_box)


### Visualize 3

In [ ]:
import pandas as pd
import numpy as np
import os
from pythreejs import *
from IPython.display import display
import ipywidgets as widgets
import asyncio

# Load data
body_names = ['Sun', 'Mercury', 'Venus', 'Earth', 'Mars', 'Jupiter', 'Saturn', 'Uranus', 'Neptune']
data = []

for name in body_names:
    df = pd.read_csv(f'outputs/{name}.csv')
    data.append(df[['X', 'Y', 'Z']].to_numpy())

num_frames = len(data[0])
num_bodies = len(data)

# Create spheres
colors = ['yellow', 'gray', 'orange', 'blue', 'red', 'brown', 'gold', 'cyan', 'purple']
radii = [0.3, 0.05, 0.06, 0.07, 0.06, 0.2, 0.17, 0.12, 0.12]

spheres = []
for i in range(num_bodies):
    mat = MeshLambertMaterial(color=colors[i])
    geo = SphereBufferGeometry(radius=radii[i], widthSegments=16, heightSegments=16)
    mesh = Mesh(geometry=geo, material=mat, position=tuple(data[i][0]))
    spheres.append(mesh)

# Create orbit trails (line geometry from all positions)
orbits = []
for i in range(num_bodies):
    positions = data[i].astype(np.float32)
    orbit_geometry = BufferGeometry(attributes={
        'position': BufferAttribute(positions, normalized=False)
    })
    orbit_line = Line(
        geometry=orbit_geometry,
        material=LineBasicMaterial(color=colors[i], linewidth=1),
        type='LineStrip'
    )
    orbits.append(orbit_line)

# Set up the scene
scene = Scene(children=[
    *spheres,
    *orbits,
    AmbientLight(intensity=0.5),
    DirectionalLight(position=[5, 5, 5], intensity=0.6)
], background='black')

camera = PerspectiveCamera(position=[0, 0, 8], fov=60)
controller = OrbitControls(controlling=camera)
renderer = Renderer(camera=camera, scene=scene, controls=[controller], width=800, height=600)

# Animation state
is_playing = False
frame_index = 0
speed = 1  # frames per update

# Update function
def update_scene():
    global frame_index
    for i in range(num_bodies):
        if frame_index < len(data[i]):
            spheres[i].position = tuple(data[i][frame_index])
    frame_index = (frame_index + speed) % num_frames

# Async animation loop
async def animate_loop():
    global is_playing
    while is_playing:
        update_scene()
        await asyncio.sleep(0.03)

# Button handlers
def toggle_play(change):
    global is_playing
    is_playing = not is_playing
    play_button.description = 'Pause' if is_playing else 'Play'
    if is_playing:
        asyncio.ensure_future(animate_loop())

def reset_scene(change):
    global frame_index
    frame_index = 0
    update_scene()

def increase_speed(change):
    global speed
    speed = min(speed + 1, 10)

def decrease_speed(change):
    global speed
    speed = max(speed - 1, 1)

# Buttons
play_button = widgets.Button(description='Play')
play_button.on_click(toggle_play)

reset_button = widgets.Button(description='Reset')
reset_button.on_click(reset_scene)

faster_button = widgets.Button(description='Faster')
faster_button.on_click(increase_speed)

slower_button = widgets.Button(description='Slower')
slower_button.on_click(decrease_speed)

controls = widgets.HBox([play_button, reset_button, slower_button, faster_button])

# Display everything
display(renderer, controls)
